# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверьте 4 гипотезы:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок.
2. Есть ли зависимость между семейным положением и возвратом кредита в срок.
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок.
4. Как разные цели кредита влияют на его возврат в срок.

**Ход исследования**

Данные получаем из файла `data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Далее, проверяем данные на ошибки и оцениваем их влияние на исследование. Затем, на этапе предобработки исправляем самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [154]:
# импорт библиотеки pandas
import pandas as pd

In [155]:
# чтение файла с данными и сохранение в df
df = pd.read_csv(r'C:\Users\Dns\Desktop\проект\data.csv')

In [156]:
# получение первых 10 строк таблицы df
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [157]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В таблице 12 столбцов. Тип данных различный, не противоречит смыслу переданной информации. 5 столбцов - 'object', 5 столбцов - 'int', 2 столбца - 'float'.

Согласно документации к данным:
* 'children' — количество детей в семье;
* 'days_employed' — общий трудовой стаж в днях;
* 'dob_years' — возраст клиента в годах;
* 'education' — уровень образования клиента;
* 'education_id' — идентификатор уровня образования;
* 'family_status' — семейное положение;
* 'family_status_id' — идентификатор семейного положения;
* 'gender' — пол клиента;
* 'income_type' — тип занятости;
* 'debt' — имел ли задолженность по возврату кредитов;
* 'total_income' — ежемесячный доход;
* 'purpose' — цель получения кредита.

Названия колонок указаны без нарушений стиля Python.
В столбце 'days_employed' есть положительные и отрицательные значения, что противоречит смыслу.
В столбце 'education' информация указана в разных регистрах. На цели исследования данная информация не влияет, но лучше привести значение к общему регистру, во избежание возможных скрытых дубликатов.
Количество значений в столбцах различается. Значит в данных есть пропущенные значения.
В столбце 'purpose' значения с одной и той же информацией указаны в различных формулировках.
В столбце 'total_income' в значении указано 6 знаков после запятой, что помешает подсчету.	

Иследуем колонки с интересующими нас данными, согласно наших целей.

In [158]:
# исследуем уникальные значения и кол-во их упоминаний для стобца 'children'
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В колонке 'children' указано количество детей клиента. Наблюдаются ошибки привнесении данных:
- не может быть минус 1 ребенок, правильно указать 1
- 20 детей, также неправдоподобно, правильнее указать 2

In [159]:
# исследуем уникальные значения и кол-во их упоминаний для стобца 'family_status'
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [160]:
# исследуем уникальные значения и кол-во их упоминаний для стобца 'family_status_id'
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

В колонке 'family_status' и 'family_status_id' указано 5 значений, которые совпадают по количеству. Принимаем данную классификацию.

In [161]:
# исследуем количество уникальных значений для стобца 'total_income'
len(df['total_income'].value_counts())

19351

Количестово уникальных значений в столбце 'total_income' говорит о том, что есть пропущенные значения.

In [162]:
# исследуем уникальные значения и кол-во их упоминаний для стобца 'debt'
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

В колонке 'debt' используется 2 значения:

0 - нет задолженности

1 - есть задолженность

In [163]:
# исследуем уникальные значения и кол-во их упоминаний для стобца 'purpose'
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Количество целей для кредита очень велико. В дальнейшем будем использовать лемматизацию, чтобы сократить список.

**Выводы:**
Изучив таблицу, определили, что необходимые нам данные содержатся в следующих колонках:

* 'children' — количество детей в семье;
* 'family_status' — семейное положение;
* 'debt' — имел ли задолженность по возврату кредитов;
* 'total_income' — ежемесячный доход;
* 'purpose' — цель получения кредита.

Данных достаточно для проверки гипотез, но есть проблемы, которые устраняем на этапе предобработки.
1. Неявные дубликаты в колонке 'children'.
2. Колонка 'total_income' имеет пропущенные значения.

Столбцы 'days_employed', 'dob_years', 'education', 'gender', 'income_type' безусловно влияют на ежемесячный доход, но в нашем исследовании данная закономерность не рассматривается.

## Шаг 2. Предобработка данных

### Обработка пропусков

#### Поиск пропущенных значений

In [164]:
# подсчет пропусков по всему датасету
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Имеем 2 колонки с пропусками:'days_employed' и 'total_income'. Изучим их подробнее. Скорее всего, при заполнении данных происходила техническая ошибка.

In [165]:
#выведем первые 10 строк столбца days_employed
df['days_employed'].head(10)

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
5      -926.185831
6     -2879.202052
7      -152.779569
8     -6929.865299
9     -2188.756445
Name: days_employed, dtype: float64

Как ранее сообщалось, значение столбца принимают положительные и отрицательные значения, что противоречит логике.

In [166]:
#выведем первые 10 строк столбца total_income
df['total_income'].head(10)

0    253875.639453
1    112080.014102
2    145885.952297
3    267628.550329
4    158616.077870
5    255763.565419
6    240525.971920
7    135823.934197
8     95856.832424
9    144425.938277
Name: total_income, dtype: float64

Исследуем диапазон столбца 'total_income', его среднее значение и медиану.

In [167]:
# узнаем минимальное значение столбца total_income
print(df['total_income'].min())

20667.26379327158


In [168]:
# узнаем максимальное значение столбца total_income
print(df['total_income'].max())

2265604.028722744


In [169]:
# узнаем среднее арифметическое значение столбца total_income
print(df['total_income'].mean())

167422.3022081719


In [170]:
# узнаем медианное значение столбца total_income
print(df['total_income'].median())

145017.93753253992


Средний доход варьируется от 20 тыс. до 2,265 миллионов рублей. Медиана и среднее примерно равны, поэтому приходим к выводу, что данные верные. Заполним пропущенные значения медианным. 

In [171]:
#заполняем пропущенные значения в столбце total_income
df['total_income'] = df['total_income'].fillna(df['total_income'].median())

Столбец 'days_employed' не влияет на исследования, принимаем решение удалить его.

In [172]:
# удаляем столбец с пропущенными значениями days_employed 
df.dropna(axis='columns', inplace = True)

In [173]:
# проверяем датасет на пропуски еще раз
df.isna().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Обнаружили пропуски в столбцах: 'total_income' и 'days_employed'. Столбец 'days_employed' удален из-за количества ошибок и отсутствия ценности, столбец 'total_income' заполнили медианным значением.  

### Замена типа данных

In [174]:
# для удобства отображения меняем тип данных в столбце total_income с float64 на int64
df['total_income'] = df['total_income'].astype('int')

**Вывод**

Данные в столбце 'total_income' заменены с вещественного типа данных на целочисленный для удобства отображения и подсчета.

### Обработка дубликатов

Перед проверкой дубликатов переведем значения в столбце 'education' в нижний регистр.

In [175]:
#  перевод значений в нижний регистр
df['education'] = df['education'].str.lower()

In [176]:
# подсчет явных дубликатов
df.duplicated().sum()

71

In [177]:
# удаляем явные дубликаты с удалением старых индексов и формированием новых
df = df.drop_duplicates().reset_index(drop=True)

In [178]:
# проверяем кол-во явных дубликатов
df.duplicated().sum()

0

Ранее обнаружили в столбце 'children' неявные дубликаты, а именно значения: -1, 20. Допустим, что верное значение: 1, 2; и произведем замену.

In [179]:
# устраняем неявный дубликат
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)
# подсчет уникальных значений
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

Обнаружили 71 явный дубликат в датасет, 2 варианта неявных дубликата в столбце 'children' и устранили их.

### Лемматизация

Выясним, каковы уникальные цели кредита с помощью лемматизации.
Импортируем из библиотеки pymystem3 функцию Mystem

In [180]:
# импорт функции лемитизации Mystem
from pymystem3 import Mystem
m = Mystem()

Создаем Series с уникальными значениями целей кредита.

In [181]:
# сохранение Series со значениями целей кредита в переменную
purpose_unique = df['purpose'].unique()

In [182]:
# создаем пустой список с лемматизированными значениями целей кредита
lemmas = []

In [183]:
#заполняем список с помощью цикла
for purpose in purpose_unique:
    lemma = m.lemmatize(purpose)
    lemmas.append(lemma)

In [184]:
# просматриваем результат
lemmas

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

**Вывод**

На основании полученных данных делаем вывод, что все цели кредита заключаются в следующих категориях: недвижимость, автомобиль, образование и свадьба.

### Категоризация данных

Ранее определили, что для целей исследования нам потребуются данные из столбцов: 'children', 'family_status', 'debt', 'total_income', 'purpose', но не все они представлены в удобном для анализе виде.
Если в столбце 'children' и'family_status' всего 5 уникальных значений, то в столбце 'total_income' значений очень много, в столбце 'purpose' их более 30-ти, хотя фактически всего 4. Необходимо произвести категоризацию данных.   
Также, для большей наглядности и удобства построения таблиц принято решение произвести категоризацию данных из столбца 'debt'.
Создадим столбец для категорий целей кредита, с помощью функции.

In [185]:
# разбиваем на категории по леммам
def purpose_category(purpose):
    if 'авто' in purpose:
        return 'автомобиль'
    elif 'образован' in purpose:
        return 'образование'
    elif 'свад' in purpose:
        return 'свадьба'
    elif 'недвиж' or 'жил' in purpose:
        return 'недвижимость'

In [186]:
# создаем столбец для новых значений категории
df['purpose_to_category'] = df['purpose'].apply(purpose_category)


In [187]:
# проверяем результат категоризации
df['purpose_to_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_to_category, dtype: int64

Создаем новую колонку для категоризации ежемесячного дохода. На основании данных мы определяем 4 категории ежемесячного дохода: "низкий доход", "средний доход", высокий доход" и "очень высокий доход". Создаем функцию, которая разобъет значения из столбца 'total_income' на 4 категории, запишем данные в отдельный столбец и далее произведем проверку, анйдя кол-во каждого значения.

In [188]:
#разбиваем ежемесячный доход на категории
def total_income_category(total_income):
    if total_income <= 65000:
        return 'низкий доход'
    elif total_income <= 130000:
        return 'средний доход'
    elif total_income <= 500000:
        return 'высокий доход'
    elif 500000 <= total_income:
        return 'очень высокий доход'

In [189]:
# создаем новый столбец для категории
df['total_income_to_category'] = df['total_income'].apply(total_income_category)

In [190]:
# проверяем результат категоризации
df['total_income_to_category'].value_counts()

высокий доход          13289
средний доход           6845
низкий доход            1098
очень высокий доход      222
Name: total_income_to_category, dtype: int64

Создадим новый столбец для категории о задолженности: 'должник', если клиент не вернул кредит вовремя и 'вернул кредит'.

In [191]:
#разбиваем сведения о задолженности на категории
def debt_category(debt):
    if debt > 0:
        return 'должник'
    else:
        return 'вернул кредит'

In [192]:
# создаем новый столбец для категории
df['debt_to_category'] = df['debt'].apply(debt_category)

In [193]:
# проверяем результат категоризации
df['debt_to_category'].value_counts()

вернул кредит    19713
должник           1741
Name: debt_to_category, dtype: int64

In [194]:
# проверяем результаты предобработки данных
df.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_to_category,total_income_to_category,debt_to_category
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий доход,вернул кредит
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний доход,вернул кредит
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,высокий доход,вернул кредит
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий доход,вернул кредит
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий доход,вернул кредит
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высокий доход,вернул кредит
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,высокий доход,вернул кредит
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,высокий доход,вернул кредит
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,средний доход,вернул кредит
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,высокий доход,вернул кредит


In [195]:
# получение общей информации о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   children                  21454 non-null  int64 
 1   dob_years                 21454 non-null  int64 
 2   education                 21454 non-null  object
 3   education_id              21454 non-null  int64 
 4   family_status             21454 non-null  object
 5   family_status_id          21454 non-null  int64 
 6   gender                    21454 non-null  object
 7   income_type               21454 non-null  object
 8   debt                      21454 non-null  int64 
 9   total_income              21454 non-null  int32 
 10  purpose                   21454 non-null  object
 11  purpose_to_category       21454 non-null  object
 12  total_income_to_category  21454 non-null  object
 13  debt_to_category          21454 non-null  object
dtypes: int32(1), int64(5),

**Вывод**

На этапе преобработки данных убрали пропущенные значения, удалили явные дубликаты, убрали неявные дубликаты, привели значения к правильным типам, провели лимитизацию и категоризацию данных.

## Шаг 3. Проверка гипотез

#### Проверка первой гипотезы
Первая гипотеза утверждает, что клиенты имеющие детей, реже гасят кредит в срок, то есть чем больше детей, тем больше вероятность непогашения кредита.
Проверим это предположение по данным о количестве детей и наличии задолжености.
Порядок действий:
* создадим сводную таблицу кол-ва должников  среди клиентов с разным кол-вом детей
* посчитаем отношение должников к общему кол-ву клиентов
* отсортируем получившееся значение по убыванию

In [196]:
# создаем сводную таблицу
df_pivot_debt_children = df.pivot_table(index='children', columns='debt_to_category', values='debt', aggfunc='count')

In [197]:
# считаем отношение должников ко всем клиентам в процентах и сохраняем в новую колонку ratio
df_pivot_debt_children['ratio'] = df_pivot_debt_children['должник'] / (df_pivot_debt_children['должник'] + df_pivot_debt_children['вернул кредит']) * 100

In [198]:
# сортируем таблицу по колонке ratio
df_pivot_debt_children.sort_values('ratio', ascending = False)

debt_to_category,вернул кредит,должник,ratio
children,,,
4,37.0,4.0,9.756098
2,1926.0,202.0,9.492481
1,4410.0,445.0,9.165808
3,303.0,27.0,8.181818
0,13028.0,1063.0,7.543822
5,9.0,NaN,NaN


**Вывод**

Наибольшее количество должников присутствует у клиентом с четырьмя детьми. Далее идут клиенты с двумя детьми, одним ребенком и тремя детьми, клиенты же без детей наиболее платежеспособные. Но так как разница незначительна, делаем вывод о том, что первая гипотеза подтвердилась лишь частично. 

#### Проверка второй гипотезы

Вторая гипотеза утверждает, что семейное положение клиента влияет на факт погашения кредита в срок, при этом клиенты, состоящие в браке, чаще гасят кредиты в срок.
Проверим это предположение по данным о количестве должников среди клиентов с разным семейным положением.
Для этого:

* создадим сводную таблицу количества должников среди клиентов с разным семейным положением
* посчитаем отношение должников к общему количеству клиентов
* отсортируем получившееся значение по убыванию

In [199]:
# создаем сводную таблицу
df_pivot_debt_family_status = df.pivot_table(index='family_status', columns='debt_to_category', values='debt', aggfunc='count')

In [200]:
# считаем отношение должников ко всем клиентам в процентах и сохраняем в новую колонку ratio
df_pivot_debt_family_status['ratio'] = df_pivot_debt_family_status['должник'] / (df_pivot_debt_family_status['должник'] + df_pivot_debt_family_status['вернул кредит']) * 100

In [201]:
# сортируем таблицу по колонке ratio
df_pivot_debt_family_status.sort_values('ratio', ascending = False)

debt_to_category,вернул кредит,должник,ratio
family_status,,,
Не женат / не замужем,2536,274,9.750890
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343


**Вывод**

Наибольшее количество должников присутствует у клиентов, которые никогда не состояли в официальном браке. В то же время меньше всего должников в категории женат/замужем, в разводе, вдовец/вдова. Вторая гипотеза подтвердилась. 

#### Проверка третьей гипотезы

Третья гипотеза утверждает, что уровень дохода клиента влияет на факт погашения кредита в срок, при этом чем выше уровень дохода клиентов, тем чаще они гасят кредиты в срок.
Проверим это предположение по данным о количестве должников среди клиентов с разным уровне дохода. Для этого:

* создадим сводную таблицу количества должников среди клиентов с разным уровнем дохода
* посчитаем отношение должников к общему количеству клиентов
* отсортируем получившееся значение по убыванию

In [202]:
# создаем сводную таблицу
df_pivot_debt_total_income_to_category = df.pivot_table(index='total_income_to_category', columns='debt_to_category', values='debt', aggfunc='count')

In [203]:
# считаем отношение должников ко всем клиентам в процентах и сохраняем в новую колонку ratio
df_pivot_debt_total_income_to_category['ratio'] = df_pivot_debt_total_income_to_category['должник'] / (df_pivot_debt_total_income_to_category['должник'] + df_pivot_debt_total_income_to_category['вернул кредит']) * 100

In [204]:
# сортируем таблицу по колонке ratio
df_pivot_debt_total_income_to_category.sort_values('ratio', ascending = False)

debt_to_category,вернул кредит,должник,ratio
total_income_to_category,,,
средний доход,6271,574,8.385683
высокий доход,12209,1080,8.127022
низкий доход,1025,73,6.648452
очень высокий доход,208,14,6.306306


**Вывод**

Клиенты с очень высоким доходом платят лучше всех, как и клиенты с низким доходом. В тоже время клиенты со средним и высоким доходом платят хуже, что может говорить о более тщательной проверке их платежеспособности. Третья гипотеза не подтвердилась.

#### Проверка четвертой гипотезы

Согласно четвертой гипотезе, цель кредита влияет на факт погашения кредита в срок, а клиенты, которые берут кредиты на покупку автомобиля, реже гасят их в срок.
Проверим это предположение по данным о количестве должников среди клиентов с разными целями кредита. Для этого:

* создадим сводную таблицу количества должников среди клиентов с разными целями кредита
* посчитаем отношение должников к общему количеству клиентов
* отсортируем получившееся значение по убыванию

In [205]:
# создаем сводную таблицу
df_pivot_debt_purpose_to_category = df.pivot_table(index='purpose_to_category', columns='debt_to_category', values='debt', aggfunc='count')

In [206]:
# считаем отношение должников ко всем клиентам в процентах и сохраняем в новую колонку ratio
df_pivot_debt_purpose_to_category['ratio'] = df_pivot_debt_purpose_to_category['должник'] / (df_pivot_debt_purpose_to_category['должник'] + df_pivot_debt_purpose_to_category['вернул кредит']) * 100

In [207]:
# сортируем таблицу по колонке ratio
df_pivot_debt_purpose_to_category.sort_values('ratio', ascending = False)

debt_to_category,вернул кредит,должник,ratio
purpose_to_category,,,
автомобиль,3903,403,9.359034
образование,3643,370,9.220035
свадьба,2138,186,8.003442
недвижимость,10029,782,7.233373


**Вывод**

Наибольшее количество не вернувших кредит вовремя, брали его на покупку автомобиля, образование, в товремя, как на самые отвественные клиенты брали кредит на жилье. Доказано, что цель кредита влияет на факт погашения кредита. Четвертая гипотеза подтвердилась.

## Шаг 4. Итоги исследования

Проведено исследование и установлено следующее:

#### 1. Количество детей у клиента частично влияет на факт погашения кредита в срок.
Наибольшее количество должников присутствует у клиентом с четырьмя детьми. Далее идут клиенты с двумя детьми, одним ребенком и тремя детьми, клиенты же без детей наиболее платежеспособные. При этом разница незначительна.
#### Первая гипотеза подтвердилась частично.
Вполне объяснимо, что клиенты не имеющие детей, не имеют постоянных дополнительных расходов и имеют большую возможность оплачивать свои кредитные обязательства. В то же время, клиенты имеющие детей, могут воспользоваться льготами от государства и различными выгодными предложениями от банка, субсидиями, мат.капиталом.

#### 2. Семейное положение клиента влияет на факт погашения кредита в срок, при этом клиенты, состоящие в браке, чаще гасят кредиты в срок.
Наибольшее количество должников присутствует у клиентов, которые никогда не состояли в официальном браке. В то же время меньше всего должников в категории женат/замужем, в разводе, вдовец/вдова.
#### Вторая гипотеза подтвердилась.
Люди, вступающие в брак, более дисциплинированные, к тому же имеют возможность обратиться к супругу за помощью, поэтому более обязательны в закрытии своих долговых обязательств.

#### 3. Уровень дохода клиента влияет на факт погашения кредита в срок.
Клиенты с очень высоким доходом платят лучше всех, как и клиенты с низким доходом. В тоже время клиенты со средним и высоким доходом платят хуже, что может говорить о более тщательной проверке их платежеспособности. 
#### Третья гипотеза не подтвердилась.
Клиенты имеющие много денежных средств, как и клиенты ограниченные в средствах, должны постоянно о них заботится, уделять им время, поэтому более тщательно следят за своевременной оплатой кредита. Клиенты имеющие средний и высокий достаток, менее щепетильны в данном вопросе.

#### 4. Цель кредита влияет на факт погашения кредита в срок, а клиенты, которые берут кредиты на покупку автомобиля, реже гасят их в срок.
Наибольшее количество задолжников брало кредит на покупку автомобиля, оплату образование; наименьшее - на покупку недвижимости. Доказано, что цель кредита влияет на факт погашения кредита.
#### Четвертая гипотеза подтвердилась.
Покупка автомобиля несет за собой постоянные траты на его обслуживание, в то время как жилье постоянно дорожает, так же может использоваться как актив, а имнно - сдача в аренду.


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.